# Wiki NBA Stat Scraper

In [94]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from itertools import chain
from collections import defaultdict
from itertools import cycle 

In [2]:
url = 'https://en.wikipedia.org/wiki/Michael_Jordan'

In [3]:
r = requests.get(url)
text = r.text
soup = BeautifulSoup(text, 'html.parser')

In [4]:
# Use prettify to get a good
# look at the HTML
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Michael Jordan - Wikipedia
  </title>
  <script>
   document.documentElement.className=document.documentElement.className.replace(/(^|\s)client-nojs(\s|$)/,"$1client-js$2");RLCONF={"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Michael_Jordan","wgTitle":"Michael Jordan","wgCurRevisionId":913677220,"wgRevisionId":913677220,"wgArticleId":20455,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Webarchive template webcite links","Wikipedia indefinitely move-protected pages","Articles with short description","Use American English from September 2019","All Wikipedia articles written in American English","Use mdy dates from September 2019","Wikipedia indefinitely semi-protected biographies of living people","Articles containing potentially dated

In [5]:
# I'm using this method to create wikitable
wikitable = soup.find('table', {'class':'wikitable sortable'})

In [6]:
wikitable

<table class="wikitable sortable" style="font-size:95%; text-align:right;">
<tbody><tr>
<th>Year
</th>
<th>Team
</th>
<th><abbr title="Games played">GP</abbr>
</th>
<th><abbr title="Games started">GS</abbr>
</th>
<th><abbr title="Minutes per game">MPG</abbr>
</th>
<th><abbr title="Field goal percentage">FG%</abbr>
</th>
<th><abbr title="3-point field-goal percentage">3P%</abbr>
</th>
<th><abbr title="Free-throw percentage">FT%</abbr>
</th>
<th><abbr title="Rebounds per game">RPG</abbr>
</th>
<th><abbr title="Assists per game">APG</abbr>
</th>
<th><abbr title="Steals per game">SPG</abbr>
</th>
<th><abbr title="Blocks per game">BPG</abbr>
</th>
<th><abbr title="Points per game">PPG</abbr>
</th></tr>
<tr>
<td style="text-align:left;"><a href="/wiki/1984%E2%80%9385_NBA_season" title="1984–85 NBA season">1984–85</a>
</td>
<td style="text-align:left;"><a href="/wiki/1984%E2%80%9385_Chicago_Bulls_season" title="1984–85 Chicago Bulls season">Chicago</a>
</td>
<td><b>82</b></td>
<td><b>82</b></

In [7]:
# Find the stat table
#wikitable = soup.find('table', class_='wikitable sortable')

In [10]:
th = wikitable.find_all(['th'])

In [15]:
th

[<th>Year
 </th>, <th>Team
 </th>, <th><abbr title="Games played">GP</abbr>
 </th>, <th><abbr title="Games started">GS</abbr>
 </th>, <th><abbr title="Minutes per game">MPG</abbr>
 </th>, <th><abbr title="Field goal percentage">FG%</abbr>
 </th>, <th><abbr title="3-point field-goal percentage">3P%</abbr>
 </th>, <th><abbr title="Free-throw percentage">FT%</abbr>
 </th>, <th><abbr title="Rebounds per game">RPG</abbr>
 </th>, <th><abbr title="Assists per game">APG</abbr>
 </th>, <th><abbr title="Steals per game">SPG</abbr>
 </th>, <th><abbr title="Blocks per game">BPG</abbr>
 </th>, <th><abbr title="Points per game">PPG</abbr>
 </th>]

In [59]:
header = []
num = []
dataset = []

In [60]:
for col in th:
    col = col.get_text()
    col = col.rstrip()
    header.append(col)

In [61]:
header

['Year',
 'Team',
 'GP',
 'GS',
 'MPG',
 'FG%',
 '3P%',
 'FT%',
 'RPG',
 'APG',
 'SPG',
 'BPG',
 'PPG']

In [62]:
# Next get each row

In [63]:
td = wikitable.find_all(['td'])

In [64]:
for row in td:
    row = row.get_text()
    row = row.rstrip()
    num.append(row)

In [82]:
# drop the last two rows
# this will be a problem in merging the data

In [74]:
x = True
a = []
while x == True:
    for cell in num:
        if cell != 'Career':
            a.append(cell)
            continue
        else:
            x = False
            break

In [85]:
np.shape(a)

(195,)

In [77]:
print(len(header))
print(len(num))
print(len(a))

13
219
195


In [147]:
# Able to create a dic of the all the vaule however the key values are backwards
# try to find a better way

In [145]:
# Initialising list of dictionary 
  
# zipping in cyclic if shorter length 
result = {v: header[i % len(header)]  
          for i, v in enumerate(num)} 
  
print("resultant dictionary : ", str(result)) 

resultant dictionary :  {'1984–85': 'Year', 'Chicago': 'Team', '82': 'GP', '38.3': 'GS', '.515': 'FG%', '.173': '3P%', '.845': 'FT%', '6.5': 'RPG', '5.9': 'RPG', '2.4': 'SPG', '.8': 'SPG', '28.2': 'PPG', '1985–86': 'Year', '18': 'GP', '7': 'GS', '25.1': 'MPG', '.457': 'FG%', '.167': '3P%', '.840': 'FT%', '3.6': 'RPG', '2.9': 'SPG', '2.1': 'SPG', '1.2': 'BPG', '22.7': 'PPG', '1986–87': 'Year', '40.0': 'MPG', '.482': 'FG%', '.182': '3P%', '.857': 'FT%', '5.2': 'APG', '4.6': 'APG', '1.5': 'SPG', '37.1*': 'PPG', '1987–88': 'Year', '40.4*': 'MPG', '.535': 'FG%', '.132': '3P%', '.841': 'FT%', '5.5': 'APG', '3.2*': 'SPG', '1.6': 'BPG', '35.0*': 'PPG', '1988–89': 'Year', '81': 'GS', '40.2*': 'MPG', '.538': 'FG%', '.276': '3P%', '.850': 'FT%', '8.0': 'APG', '32.5*': 'PPG', '1989–90': 'Year', '39.0': 'MPG', '.526': 'FG%', '.376': '3P%', '.848': 'FT%', '6.9': 'RPG', '6.3': 'APG', '2.8*': 'SPG', '.7': 'BPG', '33.6*': 'PPG', '1990–91†': 'Year', '37.0': 'MPG', '.539': 'FG%', '.312': '3P%', '.851': '

In [107]:
# Initialising list of dictionary 
  
# zipping in cyclic if shorter length 
result2 = {v: header[i % len(header)]  
          for i, v in enumerate(a)} 
  
print("resultant dictionary : ", str(result2)) 

resultant dictionary :  {'1984–85': 'Year', 'Chicago': 'Team', '82': 'GP', '38.3': 'MPG', '.515': 'FG%', '.173': '3P%', '.845': 'FT%', '6.5': 'RPG', '5.9': 'RPG', '2.4': 'SPG', '.8': 'BPG', '28.2': 'PPG', '1985–86': 'Year', '18': 'GP', '7': 'GS', '25.1': 'MPG', '.457': 'FG%', '.167': '3P%', '.840': 'FT%', '3.6': 'RPG', '2.9': 'SPG', '2.1': 'SPG', '1.2': 'BPG', '22.7': 'PPG', '1986–87': 'Year', '40.0': 'MPG', '.482': 'FG%', '.182': '3P%', '.857': 'FT%', '5.2': 'APG', '4.6': 'APG', '1.5': 'SPG', '37.1*': 'PPG', '1987–88': 'Year', '40.4*': 'MPG', '.535': 'FG%', '.132': '3P%', '.841': 'FT%', '5.5': 'APG', '3.2*': 'SPG', '1.6': 'BPG', '35.0*': 'PPG', '1988–89': 'Year', '81': 'GS', '40.2*': 'MPG', '.538': 'FG%', '.276': '3P%', '.850': 'FT%', '8.0': 'APG', '32.5*': 'PPG', '1989–90': 'Year', '39.0': 'MPG', '.526': 'FG%', '.376': '3P%', '.848': 'FT%', '6.9': 'RPG', '6.3': 'APG', '2.8*': 'SPG', '.7': 'BPG', '33.6*': 'PPG', '1990–91†': 'Year', '37.0': 'MPG', '.539': 'FG%', '.312': '3P%', '.851': 

In [ ]:
# after creating the dic properly should I turn it into a dataframe?

In [155]:
df = pd.DataFrame.from_dict(result2)# orient='index',columns=['A', 'B', 'C', 'D'])

ValueError: If using all scalar values, you must pass an index

In [72]:
df.describe()

NameError: name 'df' is not defined

In [ ]:
# create a dataset for the player stats

In [ ]:
# Repeat the same for the playoff numbers

In [58]:
# Upload in SQLite
# If play is in db do nothing
# Else upload data

In [59]:
# Create visualizations in matplotlib